In [44]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

In [98]:
df = pd.read_csv('data/flight_traffic.csv') 
df.head()

,year,month,day,airline_id,origin_airport,destination_airport,scheduled_departure,actual_departure,taxi_out,wheels_off,...,cancelled,diverted,scheduled_elapsed,actual_elapsed,distance,airline_delay,weather_delay,air_system_delay,security_delay,aircraft_delay
0,2017,1,17,AA,CLT,PHX,1619,1616.0,17.0,1633.0,...,0.0,0.0,277.0,266.0,1773.0,NaN,NaN,NaN,NaN,NaN
1,2017,1,18,AA,CLT,PHX,1619,1614.0,13.0,1627.0,...,0.0,0.0,277.0,247.0,1773.0,NaN,NaN,NaN,NaN,NaN
2,2017,1,19,AA,CLT,PHX,1619,1611.0,17.0,1628.0,...,0.0,0.0,277.0,255.0,1773.0,NaN,NaN,NaN,NaN,NaN
3,2017,1,20,AA,CLT,PHX,1619,1656.0,18.0,1714.0,...,0.0,0.0,277.0,273.0,1773.0,33.0,0.0,0.0,0.0,0.0
4,2017,1,21,AA,CLT,PHX,1619,1632.0,17.0,1649.0,...,0.0,0.0,277.0,266.0,1773.0,NaN,NaN,NaN,NaN,NaN


In [90]:
def convertDateToMinutes(t):
    t = int(t)
    hours = (t/100) % 100
    minutes = t % 100
    return 60*hours + minutes
def getTimeDifference(t1, t2):
    totalMin = 24*60
    t1 = convertDateToMinutes(t1)
    t2 = convertDateToMinutes(t2)
    if t1 > t2 and t1 - t2 < 60: # early departure
        return t2 - t1
    return (t2 - t1) % totalMin

In [92]:
print getTimeDifference(510, 1510)
print getTimeDifference(2230, 130)
print getTimeDifference(600, 555)

600
180
-5


In [100]:
lax_df = df.loc[df['origin_airport'] == 'LAX']
print "LAX", lax_df.shape
lax_df = lax_df[lax_df['scheduled_departure'].notnull()]
lax_df = lax_df[lax_df['actual_departure'].notnull()]
lax_df = lax_df[lax_df['scheduled_arrival'].notnull()]
lax_df = lax_df[lax_df['actual_arrival'].notnull()]
lax_df = lax_df[lax_df['scheduled_elapsed'].notnull()]
lax_df = lax_df[lax_df['actual_elapsed'].notnull()]
print "LAX after filter Nan", lax_df.shape

LAX (214297, 24)
LAX after filter Nan (211730, 24)


In [101]:
lax_df.loc[:,'dep_diff'] = lax_df.apply(lambda row: getTimeDifference(row['scheduled_departure'], row['actual_departure']), axis=1)
lax_df.loc[:,'arr_diff'] = lax_df.apply(lambda row: getTimeDifference(row['scheduled_arrival'], row['actual_arrival']), axis=1)
lax_df.loc[:,'elapsed_diff'] = lax_df['actual_elapsed'] - lax_df['scheduled_elapsed']
lax_df.head()

,year,month,day,airline_id,origin_airport,destination_airport,scheduled_departure,actual_departure,taxi_out,wheels_off,...,actual_elapsed,distance,airline_delay,weather_delay,air_system_delay,security_delay,aircraft_delay,dep_diff,arr_diff,elapsed_diff
101,2017,1,7,AA,LAX,PHX,600,555.0,21.0,616.0,...,75.0,370.0,NaN,NaN,NaN,NaN,NaN,-5,-20,-15.0
169,2017,1,1,AA,LAX,OMA,1820,1813.0,16.0,1829.0,...,174.0,1330.0,NaN,NaN,NaN,NaN,NaN,-7,-31,-24.0
170,2017,1,2,AA,LAX,OMA,1820,1815.0,18.0,1833.0,...,171.0,1330.0,NaN,NaN,NaN,NaN,NaN,-5,-32,-27.0
171,2017,1,3,AA,LAX,OMA,1820,1815.0,14.0,1829.0,...,157.0,1330.0,NaN,NaN,NaN,NaN,NaN,-5,-46,-41.0
172,2017,1,4,AA,LAX,OMA,1820,1815.0,22.0,1837.0,...,188.0,1330.0,NaN,NaN,NaN,NaN,NaN,-5,-15,-10.0
